<font size=6>🥳**欢迎体验Jupyter云上开发环境**</font>

# 使用须知
> - 目前`jupyter`运行时长限制只能运行2个小时。**2个小时之后将会自动释放所有资源，请提前将所需资源下载到本地或git push到平台中**。
> - 本平台是开放的学习平台，禁止使用本平台进行商业用途和非法用途或者恶意攻击，一旦发现将依法追究。

此开发环境是使用的是`Jupyter Lab 3.x`，并集成了实用的插件。您可以在`Jupyter Lab`上灵活运行调试代码和编写文档，`Jupyter Lab`集成了很多编辑器，例如 `Jupyter` 笔记本、文本编辑器、终端和自定义组件。更多的详细介绍你可以查看 [JupyterLab官网文档介绍](https://jupyterlab.readthedocs.io/en/stable/getting_started/overview.html#overview)。
    
接下来跟我来一起沉浸式地探索`Jupyter Lab`吧❗

**若您不熟悉`Jupyter Lab`, 您可以先阅读`Jupyter操作文档.inynb`文件熟悉`Jupyter`的基本操作**

# 快速体验平台操作

## 快速体验推理
平台集成了[大模型套件MindSpore MindFormers](https://gitee.com/mindspore/mindformers/tree/r0.3/)，可支持下载和加载类Transformer和SOTA模型，并支持pipeline推理，目前支持text classifcation、token classification、swin、vit、zero shot classification和question answering等下游任务。
未来平台将会集成MindSpore MindFormers更多的特性，**MindFormers具有更多的特性，比如预训练、微调、评估、多卡训练等，如果感兴趣可以进入下面代码仓，一键三连哦**。
- [MindFormers仓](https://gitee.com/mindspore/mindformers/tree/r0.3/)
- [MindSPore仓](https://gitee.com/mindspore/mindspore)

**MindFormers下游任务体验链接（持续更新中）**：
- [vit](https://xihe.mindspore.cn/projects/MindSpore/vit_image_classification)
- [swin](https://xihe.mindspore.cn/projects/MindSpore/Swin)
- [token classification](https://xihe.mindspore.cn/projects/MindSpore/token_classification)
- [text classifcation](https://xihe.mindspore.cn/projects/MindSpore/text_classification)
- [zero shot classification](https://xihe.mindspore.cn/projects/MindSpore/zero_shot_image_classification)
- [question answering](https://xihe.mindspore.cn/projects/MindSpore/question_answering)

**你可以`Ctrl+Enter`运行下面代码，或者使用菜单栏的三角形运行下面的Cell，注意一定要按顺序执行。**

In [ ]:
# 若出现ValueError: Failed to read the checkpoint file . May not have permission to read it, please check the correct of the file. 请将下面注释放开
# !rm checkpoint_download/vit/ -rf
from mindformers.pipeline import pipeline
from mindformers.tools.image_tools import load_image

pipeline_task = pipeline("image_classification", model="mindspore/vit_base_p16")
img = load_image(
    "https://ascend-repo-modelzoo.obs.cn-east-2.myhuaweicloud.com/XFormer_for_mindspore/clip/sunflower.png"
)
pipeline_result = pipeline_task(img, top_k=3)
pipeline_result

In [ ]:
# 若出现ValueError: Failed to read the checkpoint file . May not have permission to read it, please check the correct of the file. 请将下面注释放开
# !rm checkpoint_download/vit/ -rf
from mindformers import AutoConfig, AutoTokenizer, BertForTokenClassification
from mindformers.dataset.labels import cluener_labels
from mindformers.pipeline import TokenClassificationPipeline

input_data = ["表身刻有代表日内瓦钟表匠freresoltramare的“fo”字样。"]

id2label = {label_id: label for label_id, label in enumerate(cluener_labels)}

tokenizer = AutoTokenizer.from_pretrained(
    "mindspore/tokcls_bert_base_chinese_cluener",
)
tokcls_cluener_config = AutoConfig.from_pretrained(
    "mindspore/tokcls_bert_base_chinese_cluener",
)

# This is a known issue, you need to specify batch size equal to 1 when creating bert model.
tokcls_cluener_config.batch_size = 1

model = BertForTokenClassification(tokcls_cluener_config)
tokcls_pipeline = TokenClassificationPipeline(
    task="token_classification",
    model=model,
    id2label=id2label,
    tokenizer=tokenizer,
    max_length=model.config.seq_length,
    padding="max_length",
)

results = tokcls_pipeline(input_data)
results

## 快速体验训练
**这里以Lenet5实现手写数字体为案例**

**基本介绍**

LeNet描述
LeNet是1998年提出的一种典型的卷积神经网络。它被用于数字识别并取得了巨大的成功。

[论文](https://gitee.com/link?target=https%3A%2F%2Fieeexplore.ieee.org%2Fdocument%2F726791)： Y.Lecun, L.Bottou, Y.Bengio, P.Haffner.Gradient-Based Learning Applied to Document Recognition.Proceedings of the IEEE.1998.

**模型架构**

LeNet非常简单，包含5层，由2个卷积层和3个全连接层组成。

![Lenet5](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/r1.7/tutorials/source_zh_cn/beginner/images/lenet.png)

### 下载预训练模型
下载完成后，下载完成后的目录结构如下：  

```
└── lenet_mnist
    ├── best.ckpt
    ├── lenet_ascend_v150_mnist_official_cv_acc98.49.ckpt
    ├── lenet_ascend_v160_mnist_official_cv_acc98.49.ckpt
    ├── lenet_ascend_v170_mnist_official_cv_acc98.49.ckpt
    ├── lenet-1_1875.ckpt
    └── README.md
```
这里用lenet-1_1875.ckpt做finetune

In [ ]:
!git clone https://source-xihe.mindspore.cn/MindSpore/lenet_mnist.git

###  下载MNIST数据集

下载完成后，下载完成后的目录结构如下：  
```
└── MNIST_Data
    ├── test # 测试集
    │   ├── t10k-images-idx3-ubyte 
    │   └── t10k-labels-idx1-ubyte 
    │── train # 训练集
    │    ├── train-images-idx3-ubyte
    │    └── train-lables-idx1-ubyte
    └── README.md
```

In [ ]:
from download import download

url = (
    "https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/"
    "notebook/datasets/MNIST_Data.zip"
)
path = download(url, "./", kind="zip", replace=True)

###  执行训练代码

代码来源于[lenet5代码仓](https://xihe.mindspore.cn/projects/MindSpore/lenet5/tree)的[train.py文件](https://xihe.mindspore.cn/projects/MindSpore/lenet5/blob/train/train.py)

**您可以手动地调整下面的超参数**

In [ ]:
# Copyright 2022 Huawei Technologies Co., Ltd
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ============================================================================

""" LeNet training """

import os

import mindspore
from mindspore import load_checkpoint, load_param_into_net, nn
from mindspore.train import Model
from mindspore.train.callback import CheckpointConfig, ModelCheckpoint
from mindvision.classification.dataset import Mnist
from mindvision.classification.models import lenet
from mindvision.engine.callback import LossMonitor

data_url = "./MNIST_Data"
pretrain_url = "lenet_mnist/lenet-1_1875.ckpt"
output_path = "./output"

# 超参数设置
learning_rate = 0.01
momentum = 0.9
batch_size = 32
num_epochs = 5


def train():
    # 将模型参数存入parameter的字典中，这里加载的是上面训练过程中保存的模型参数
    param_dict = load_checkpoint(pretrain_url)

    # 重新定义一个LeNet神经网络
    network = lenet(num_classes=10, pretrained=False)

    # 将参数加载到网络中
    load_param_into_net(network, param_dict)
    # 定义损失函数
    net_loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction="mean")
    # 定义优化器函数
    net_opt = nn.Momentum(
        network.trainable_params(),
        learning_rate=learning_rate,
        momentum=momentum,
    )
    model = Model(
        network,
        loss_fn=net_loss,
        optimizer=net_opt,
        metrics={"accuracy"},
    )

    # 定义训练数据集
    download_train = Mnist(
        path=data_url,
        split="train",
        batch_size=batch_size,
        repeat_num=1,
        shuffle=True,
        resize=32,
        download=True,
    )
    dataset_train = download_train.run()
    # print(len(download_train))
    # print(dataset_train.)

    # 设置模型保存参数
    config_ck = CheckpointConfig(
        save_checkpoint_steps=int(60000 / batch_size), keep_checkpoint_max=1
    )

    # 应用模型保存参数
    check_point = ModelCheckpoint(
        prefix="lenet", directory=output_path, config=config_ck
    )

    # 训练网络模型
    model.train(
        num_epochs, dataset_train, callbacks=[check_point, LossMonitor(0.01, 1875)]
    )


# 执行训练
train()

## 快速体验推理可视化
平台集成[Gradio](https://www.gradio.app/docs), 您可以自定义应用。由于安全问题你需要将`gr.xx.launch(share=True)`的`share`设为`True`，即可在Jupyter体验Gradio

In [ ]:
import cv2
import gradio as gr
import numpy as np
from mindspore import Tensor, load_checkpoint, load_param_into_net
from mindspore.nn import Softmax
from mindspore.train import Model
from mindvision.classification.models import lenet

NUM_CLASS = 10
ckpt_path = "./lenet_mnist/best.ckpt"

# 加载网路
# 将模型参数存入parameter的字典中
param_dict = load_checkpoint(ckpt_path)

# 重新定义一个LeNet神经网络,注意输入是32*32，loss采用的是SoftmaxCE
network = lenet(num_classes=NUM_CLASS, pretrained=False)
# 将参数加载到网络中
load_param_into_net(network, param_dict)
model = Model(network)


def predict_digit(img):
    # 处理图片,转化为 N，C，H,W
    img = cv2.resize(img, (32, 32))
    img = img.astype(np.float32)
    img = img / 255
    img = img.reshape((1, 1, 32, 32))

    # 预测
    predict_score = model.predict(Tensor(img)).reshape(-1)
    predict_probability = Softmax()(predict_score)

    return {str(i): predict_probability[i].asnumpy().item() for i in range(NUM_CLASS)}


gr.Interface(
    fn=predict_digit,
    inputs="sketchpad",
    outputs=gr.outputs.Label(num_top_classes=NUM_CLASS, label="预测类别"),
    live=False,
    css=".footer {display:none !important}",
    title="0-9数字画板",
    description="画0-9数字",
).launch(share=True)

## TinyMS版本推理
TinyMS是一款主要用PyThon语言编写的开源深度学习开发工具包，基于以MindSpore为代表的新型开源深度学习框架，提供面向从数据准备到模型部署全流程的极简易用的高阶API封装，并通过易于扩展的模块化设计，提供覆盖多种业务场景的能力。

相关链接
- [TinyMS官方文档](https://tinyms.readthedocs.io/en/latest/quickstart/overview.html)
- [TinyMS官方仓](https://github.com/tinyms-ai/tinyms)

In [ ]:
# 你也可以通过download下载
from download import download

url = "https://obs-xihe-beijing4.obs.cn-north-4.myhuaweicloud.com/xihe-img/projects/cloud/5.png"
path = download(url, "./5.png", kind="file", replace=True)

In [ ]:
import tinyms as ts
from mindspore import Tensor
from PIL import Image
from tinyms.model import Model, lenet5
from tinyms.vision import mnist_transform

img = Image.open("./5.png")
img = mnist_transform(img)

net = lenet5(class_num=10)
model = Model(net)
model.load_checkpoint("./lenet_mnist/best.ckpt")

input = ts.expand_dims(ts.Tensor(img), 0)
res = model.predict(input).asnumpy()
print("The label is:", mnist_transform.postprocess(res))

## 更多体验持续更新中...